In [1]:
import pandas
import datetime

/usr/local/lib/python2.7/dist-packages/pandas/_libs/__init__.py:4: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from .tslib import iNaT, NaT, Timestamp, Timedelta, OutOfBoundsDatetime
/usr/local/lib/python2.7/dist-packages/pandas/__init__.py:26: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import (hashtable as _hashtable,
/usr/local/lib/python2.7/dist-packages/pandas/core/dtypes/common.py:6: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import algos, lib
/usr/local/lib/python2.7/dist-packages/pandas/core/util/hashing.py:7: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  from pandas._libs import hashing, tslib
/usr/local/lib/python2.7/dist-packages/pandas/core/indexes/base.py:7: RuntimeWarning: numpy.dtype size changed, may indicate bi

In [2]:
sdFName = "../data/raw_cvw_20130317.txt"

In [3]:
fp = open(sdFName)

line = fp.readline()
cnt = 1
# empty lists to hold the fitted parameters
gatesList, velList, gsctList, bmList, dtList = [], [], [], [], []
glatList, glonList, gazmList, mlatList, mlonList, mazmList = [], [], [], [], [], []
# read until the end of the file
while line:
    # split the lines by whitespace
    cols = line.split()
    # check for first header line
    if cnt == 1:
        d, t = cols[0], cols[1]
        # parse the line into a datetime object
        time = datetime.datetime(int(d[:4]), int(d[5:7]), int(d[8:]),
                           int(t[:2]), int(t[3:5]), int(t[6:]))
    # check for second header line
    elif cnt == 2:
        bmnum = int(cols[2])
    # check for third header line
    elif cnt == 3:
        npnts = int(cols[2])
        nrang = int(cols[5])
        channel = cols[8]
        cpid = int(cols[11])
    # check for fourth header line
    elif cnt == 4:
        # read all of the reange gates
        for i in range(npnts):
            line = fp.readline()
            cols = line.split()
            # parse the line into the corresponding lists
            gatesList.append(int(cols[0]))
            velList.append(float(cols[5]))
            gsctList.append(int(cols[6]))
            glatList.append(float(cols[9]))
            glonList.append(float(cols[10]))
            gazmList.append(float(cols[11]))
            mlatList.append(float(cols[12]))
            mlonList.append(float(cols[13]))
            mazmList.append(float(cols[14]))
            bmList.append(bmnum)
            dtList.append(time)
        # read the blank line after each record
        line = fp.readline()
        # reset the count
        cnt = 0

    # read the next line
    line = fp.readline()
    cnt += 1

    # close the file
fp.close()

In [4]:
sdDF = pandas.DataFrame( {
    "date" : dtList,
    "beam" : bmList,
    "gate" : gatesList,
    "vel" : velList,
    "gsct" : gsctList,
    "glat" : glatList,
    "glon" : glonList,
    "gazm" : gazmList,
    "mlat" : mlatList,
    "mlon" : mlonList,
    "mazm" : mazmList
} )

In [5]:
sdDF.head()

,beam,date,gate,gazm,glat,glon,gsct,mazm,mlat,mlon,vel
0,18,2013-03-17 00:00:12,28,-1.64,55.17,-120.44,1,-11.58,61.38,-60.37,4.6
1,18,2013-03-17 00:00:12,29,-1.59,55.58,-120.46,1,-11.59,61.77,-60.54,-17.2
2,18,2013-03-17 00:00:12,32,-1.45,56.80,-120.52,1,-11.69,62.93,-61.06,9.4
3,18,2013-03-17 00:00:12,33,-1.40,57.21,-120.54,1,-11.74,63.32,-61.24,-3.7
4,18,2013-03-17 00:00:12,34,-1.36,57.62,-120.56,1,-11.80,63.70,-61.42,-9.2


In [6]:
# save to csv and feather
fName = "../data/sd-params-" + pandas.to_datetime(sdDF["date"].values[0]).strftime("%Y%m%d")
sdDF.to_csv(fName + ".csv",index=False)
sdDF.to_feather(fName + ".feather")